In [1]:
#! pip install cherrypy

In [2]:
#! pip install markupsafe==2.0.1

In [3]:
#! pip install itsdangerous==2.0.1

In [4]:
#! pip install -U Werkzeug==0.16.0

In [5]:
import plotly.express as px
import pandas as pd
import numpy as np

In [6]:
team_list = []
global team_list
modulo_scelto = '3-4-3'
global modulo_scelto 

In [7]:
player_info = pd.read_csv('../dataset/stat_info^(Stagione 2021_22).csv',index_col=[0])
player_hist = pd.read_csv('../dataset/player_history^(Stagione 2021_22).csv',index_col=[0])

player_hist.Voto = player_hist.Voto.astype(float)
player_hist.FV = player_hist.FV.astype(float)
player_hist.Bonus = player_hist.Bonus.astype(float)
player_hist.Malus = player_hist.Malus.astype(float)

all_names = player_info.Nome.unique().tolist()

classifica = pd.read_csv('../dataset/classifica_xg_xa^(Stagione 2021_22).csv',index_col=[0])

In [8]:
features = {
            
                'P': ['img','Nome','Squadra','Presenza','Gol subiti','Gol subiti 90','Rigori parati','Ammonizioni','Espulsioni',
                      'media_voto', 'media_fantavoto',
                      '%_con_voto','%_bonus', '%_malus',
                      '%_voto_<_5', '%_voto_5_6', '%_voto_6_7', '%_voto_7_8','%_voto_>_8', 
                      '%_fv_5', '%_fv_5_6', '%_fv_6_7', '%_fv_7_8', '%_fv_>_8'],
        
                'D':  [ 'img','Nome','Squadra', 'Presenza','Gol', 'Assist',
                      'Titolare', 'Entrato',
                      'Rigori calciati','Rigori segnati','Ammonizioni','Espulsioni',
                       'media_voto', 'media_fantavoto',
                       '%_con_voto','%_bonus', '%_malus',
                       'Bonus','Malus',
                       'xG', 'xA','xG90', 'xA90','Performance_gol_segnati',
                       '%_voto_<_5', '%_voto_5_6', '%_voto_6_7', '%_voto_7_8','%_voto_>_8', 
                       '%_fv_5', '%_fv_5_6', '%_fv_6_7', '%_fv_7_8', '%_fv_>_8'],
        
                'C':  [ 'img','Nome','Squadra', 'Presenza','Gol', 'Assist', 'Gol 90',
                      'Titolare', 'Entrato',
                      'Rigori calciati','Rigori segnati','Ammonizioni','Espulsioni',
                       'media_voto', 'media_fantavoto',
                       '%_con_voto','%_bonus', '%_malus',
                       'Bonus','Malus',
                       'xG', 'xA','xG90', 'xA90','Performance_gol_segnati',
                       '%_voto_<_5', '%_voto_5_6', '%_voto_6_7', '%_voto_7_8','%_voto_>_8', 
                       '%_fv_5', '%_fv_5_6', '%_fv_6_7', '%_fv_7_8', '%_fv_>_8'],
        
                'A': [ 'img','Nome','Squadra', 'Presenza','Gol', 'Assist', 'Gol 90',
                      'Titolare', 'Entrato',
                      'Rigori calciati','Rigori segnati','Ammonizioni','Espulsioni',
                       'media_voto', 'media_fantavoto',
                       '%_con_voto','%_bonus', '%_malus',
                       'Bonus','Malus',
                       'xG', 'xA','xG90', 'xA90','Performance_gol_segnati',
                       '%_voto_<_5', '%_voto_5_6', '%_voto_6_7', '%_voto_7_8','%_voto_>_8', 
                       '%_fv_5', '%_fv_5_6', '%_fv_6_7', '%_fv_7_8', '%_fv_>_8'],
               }

In [9]:
map_features_role_exploration={
    'Presenza':'Pres',
    'Gol':'Gol',
    'Assist':'Ass',
    'xG':'xGol', 
    'xA':'xAss',
    'Gol 90':'Gol90',
    'Gol subiti':'Gol subiti',
    'Gol subiti 90':'Gol subiti90',
    'Rigori parati':'Rp',
    'Titolare':'Tit',
    'Entrato':'Ent',
    'Rigori calciati':'Rc',
    'Rigori segnati':'Rs',
    'Ammonizioni':'Amm',
    'Espulsioni':'Esp',
    'media_voto' : 'MV', 
    'media_fantavoto':'MFV',
    '%_con_voto' : '%Voto',
    '%_bonus' : '%Bonus', 
    '%_malus' : '%Malus',
    'Bonus' : 'Bonus',
    'Malus' : 'Malus',
    'Performance_gol_segnati' : 'Perf',
    '%_voto_<_5': '% V<5', 
    '%_voto_5_6': '% 5<=V<6',  
    '%_voto_6_7': '% 6<=V<7', 
    '%_voto_7_8': '% 7<=V<8', 
    '%_voto_>_8': '% V>8', 
    '%_fv_5': '% FV<5', 
    '%_fv_5_6': '% 5<=FV<6',  
    '%_fv_6_7': '% 6<=FV<7', 
    '%_fv_7_8': '% 7<=FV<8', 
    '%_fv_>_8': '% FV>8', 
}

In [10]:
cast_to_int_features = ['Amm','Esp','Rs','Rc','Tit','Ent','Rp','Rs','P','Gs','A','G']

In [11]:
def view_role_explorer(role = 'P', filter_presence = 0):
    
    role_table = player_info.loc[player_info.Ruolo == role]
    role_table = role_table.loc[role_table.Presenza.fillna(0).astype(int)>int(filter_presence)]
    table = role_table[features[role]] 
    table = table.rename(columns = map_features_role_exploration)

    for feature in table.columns.tolist():
        if feature in cast_to_int_features:
            table[feature] = table[feature].astype(int)

    return render_template( 'role_explorer.html', table = table)

In [12]:
def view_builded_team(team_dict):
    
    team_df = player_info.loc[player_info.Nome.isin(team_dict)]
    
    counts_role = team_df.Ruolo.value_counts()
    
    n_dif = counts_role.loc['D']
    n_cen = counts_role.loc['C']
    n_att = counts_role.loc['A']
    
    dif_class = 'dif_'+ str(n_dif)
    cen_class = 'cen_'+ str(n_cen)
    att_class = 'att_'+ str(n_att)
    
    role_total = team_df.groupby('Ruolo')[['Bonus','Malus','Gol','Assist','xG','xA']].sum()
    role_mean = team_df.groupby('Ruolo')[['media_voto','media_fantavoto','%_con_voto', '%_fv_5', '%_fv_5_6', '%_fv_6_7', '%_fv_7_8', '%_fv_>_8',]].mean()
    
 
    table = role_total.merge(role_mean, left_index=True,right_index=True).loc[['P','D','C','A']].reset_index()
    table = np.round(table,2)
    
    return render_template( 'builded_team.html', table = table)

In [13]:
def view_team_builder(modulo_scelto):
    
    grid_list = modulo_scelto.split('-')
    dif_class = 'dif_'+grid_list[0]
    cen_class = 'cen_'+grid_list[1]
    att_class = 'att_'+grid_list[2]

    n_dif = int(grid_list[0])
    n_cen = int(grid_list[1])
    n_att = int(grid_list[2])

    nomi_dif = ['nome_' + str(2+i) for i in range(0,n_dif)]
    nomi_cen = ['nome_' + str(n_dif+2+i) for i in range(0,n_cen)]
    nomi_att = ['nome_' + str(n_dif+n_cen+2+i) for i in range(0,n_att)]

    por_names = player_info.loc[player_info.Ruolo == 'P'].Nome.unique()
    dif_names = player_info.loc[player_info.Ruolo == 'D'].Nome.unique()
    cen_names = player_info.loc[player_info.Ruolo == 'C'].Nome.unique()
    att_names = player_info.loc[player_info.Ruolo == 'A'].Nome.unique()

    return render_template( 'team_builder.html',  modulo_scelto = modulo_scelto,
                           por_names = por_names, dif_names = dif_names, cen_names = cen_names, att_names = att_names, 
                           dif_class=dif_class, cen_class=cen_class, att_class=att_class,
                           n_dif = n_dif,n_cen = n_cen,n_att = n_att,
                           nomi_dif= nomi_dif,
                           nomi_cen = nomi_cen,
                           nomi_att = nomi_att
                          )

In [14]:
def view_classifica():
    return render_template( 'classifica.html',  table =  classifica )

In [15]:
def view_player_page(nome):
    
    #extract information from the player
    data =  player_info.loc[player_info.Nome == nome]
    data = data.T.to_dict()[data.index.values[0]]
    history_data = player_hist.loc[player_hist.Nome == nome]
    history_con_voto = history_data.loc[history_data.ha_voto]
    
    #plot generation
    tmp_fantavoto = history_data[['Giornata','FV']].rename(columns={'FV':'Voto'}).reset_index(drop=True)
    tmp_fantavoto['Legenda'] = 'Fantavoto'
    tmp_voto = history_data[['Giornata','Voto']].reset_index(drop=True)
    tmp_voto['Legenda'] = 'Voto'
    
    plot_history = pd.concat([tmp_voto,tmp_fantavoto])
    plot_bonus = history_con_voto.groupby('Bonus').count().reset_index()
    
    
    fig_history = px.line(plot_history, x="Giornata", y="Voto", color="Legenda")
    plot_history = fig_history.to_html()
    
    fig = px.bar(plot_bonus, x="Bonus", y="Giornata")
    fig = fig.update_layout(yaxis_range=[0,history_data.shape[0]])
    plot_bonus = fig.to_html()
    
    ##
    return render_template('player_page.html', names = all_names,
                                               nome = data['Nome'], 
                                               team = data['Squadra'],
                                               img_src = data['img'],
                                               ruolo_classic = data['Ruolo'],
                                               media_voto = data['media_voto'],
                                               media_fantavoto = data['media_fantavoto'],
                                               mediana_voto = data['mediana_voto'],
                                               mediana_fantavoto = data['mediana_fantavoto'],
                                               presenze = int(data['Presenza']),
                                               presenze_voto = data['partite_voto'],
                                               perc_match_voto = data['%_con_voto'],
                                               minuti_giocati = int(data['Minuti']),
                                               minuti_medi = data['minuti_medi'],
                                               titolare = int(data['Titolare']),
                                               entrato = int(data['Entrato']),
                                               perc_entrato = data['%_subentrato'],
                                               sostituito = int(data['sostituito']),
                                               perc_sostituito = data['%_sostituito'],
                                               infortunato = int(data['Infortunato']),
                                               inutilizzato = int(data['Inutilizzato']),
                                               gol_segnati = data['Gol'],
                                               gol_segnati_90 = data['Gol 90'],
                                               assist = data['Assist'],
                                               xG = data['xG'],
                                               xA = data['xA'],
                                               xG90 = data['xG90'],
                                               xA90 = data['xA90'],
                                               rigori_segnati = data['Rigori segnati'],
                                               rigori_calciati = data['Rigori calciati'],
                                               status = data['Performance_gol_segnati'],
                                               totale_bonus = data['Bonus'],
                                               totale_malus = data['Malus'],
                                               partite_bonus = data['partite_bonus'],
                                               perc_bonus_match = data['%_bonus'],
                                               perc_fv_5 = data['%_fv_5'],
                                               perc_fv_5_6 = data['%_fv_5_6'],
                                               perc_fv_6_7 = data['%_fv_6_7'], 
                                               perc_fv_7_8 = data['%_fv_7_8'], 
                                               perc_fv_8 = data['%_fv_>_8'],                                               
                                               history_plot = plot_history,
                                               bonus_plot = plot_bonus)

In [ ]:
from flask import Flask, render_template,request, redirect, url_for

app = Flask(__name__)

@app.route("/")
def home_page():
    return render_template('home_page.html')

@app.route("/player-page")
def player_page(nome=all_names[0]):
    return view_player_page(nome)
    
@app.route('/player-page', methods=['POST'])
def handle_name():
    nome_selected= request.form.get('nome_selezionato')
    return view_player_page(nome_selected)

@app.route('/classifica', methods=("POST", "GET"))
def classfica_page():
    return view_classifica()

@app.route('/role-explorer')
def role_page():
    return view_role_explorer()

@app.route('/role-explorer', methods=['POST'])
def handle_filter():
    min_pres = request.form.get('min_pres')
    role_selected = request.form.get('role_selected')
    return view_role_explorer(role = role_selected, filter_presence = min_pres)

@app.route('/team-builder')
def team_builder():
    return view_team_builder(modulo_scelto)
            
@app.route('/team-builder', methods=['POST'])
def handle_team_builder():
    team_list = []
    if 'scegli_modulo' in request.form:
        modulo_scelto = request.form.get('modulo_scelto')
        return view_team_builder(modulo_scelto)
    
    if 'build_team' in request.form:
        for i in range(1,12):
            team_list.append(request.form.get('nome_'+str(i)))
        return view_builded_team(team_list)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2022 17:49:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 17:49:05] "GET /team-builder HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 17:49:06] "POST /team-builder HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 17:49:12] "GET /team-builder HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 17:49:17] "POST /team-builder HTTP/1.1" 200 -
